### Fundamentals of Natural Language Processing
# Negation and Uncertainty Detection using a Machine-Learning Based Approach

*Authors:*

> *Anna Blanco, Agustina Lazzati, Stanislav Bultaskii, Queralt Salvadó*

*Aims:*
> Our goal is to train various Machine Learning based models for each of the two sub-tasks (detection of negation and uncertainty signals, and detection of the negation and uncertainty scopes).

---

In [22]:
# Import necessary libraries and functions
import json
import pandas as pd
from utils import preprocess_text

## Data Preprocessing


In [23]:
import json
import pandas as pd

def load_data(path):
    """Load JSON data and return raw dict"""
    with open(path) as f:
        return json.load(f)

def preprocess_data(data):
    """Extract texts and predictions from loaded data"""
    texts = []
    predictions = []

    for i in range(len(data)):
        sentence = preprocess_text(data['data'][i]['text'])
        texts.append(sentence)
        predictions.append(data['predictions'][i])

    return texts, predictions

# Load train and test data
train_data = load_data('negacio_train_v2024.json')
test_data = load_data('negacio_test_v2024.json')

# Convert data into a DataFrame for simplicity
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)

# Preprocess it and obtain both the texts and the predictions
train_texts, train_preds = preprocess_data(train_df)
test_texts, test_preds = preprocess_data(test_df)

# Visualize data
print(train_texts[0])
print(train_preds[0])

nº historia clinica: nºepisodi: sexe: home data de naixement: 16.05.1936 edat: 82 anys procedencia cex mateix hosp servei urologia data d'ingres 24.07.2018 data d'alta 25.07.2018 08:54:04 ates per , ; , informe d'alta d'hospitalitzacio motiu d'ingres paciente que ingresa de forma programada para realizacion de uretrotomia interna . antecedents alergia a penicilina y cloramfenicol . no habitos toxicos. antecedentes medicos: bloqueo auriculoventricular de primer grado hipertension arterial. diverticulosis extensa insuficiencia renal cronica colelitiasis antecedentes quirurgicos: exeresis de lesiones cutaneas con anestesia local protesis total de cadera cordectomia herniorrafia inguinal proces actual varon de 81a que a raiz de episodio de hematuria macroscopica se realiza cistoscopia que es negativa para lesiones malignas pero se objetiva estenosis de uretra . se intentan dilataciones progresivas en el gabinete de urologia sin exito. se solicita estudio de imagen que confirma la existenci